In [10]:
import pandas as pd
from get_lovecraft_txt import get_and_save_remote_text, FILE_NAME
import numpy
import sys
import tensorflow
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from tqdm.notebook import tqdm

In [4]:
get_and_save_remote_text()
file = open(FILE_NAME)

In [3]:
def tokenize_words(input):
    with tqdm(total=100) as pbar:
        # lowercase everything to standardize it
        input = input.lower()
        pbar.update(10)
        # instantiate the tokenizer
        tokenizer = RegexpTokenizer(r'\w+')
        tokens = tokenizer.tokenize(input)
        pbar.update(50)
        # if the created token isn't in the stop words, make it part of "filtered"
        filtered = filter(lambda token: token not in stopwords.words('english'), tokens)
        pbar.update(40)
        return " ".join(filtered)

In [5]:
# preprocess the input data, make tokens
processed_inputs = tokenize_words(file.read())


In [6]:
chars = sorted(list(set(processed_inputs)))
char_to_num = dict((c, i) for i, c in enumerate(chars))

In [7]:
input_len = len(processed_inputs)
vocab_len = len(chars)
print ("Total number of characters:", input_len)
print ("Total vocab:", vocab_len)

Total number of characters: 2645668
Total vocab: 63


In [8]:
seq_length = 100
x_data = []
y_data = []

In [11]:
# loop through inputs, start at the beginning and go until we hit
# the final character we can create a sequence out of
for i in tqdm(range(0, input_len - seq_length, 1)):
    # Define input and output sequences
    # Input is the current character plus desired sequence length
    in_seq = processed_inputs[i:i + seq_length]

    # Out sequence is the initial character plus total sequence length
    out_seq = processed_inputs[i + seq_length]

    # We now convert list of characters to integers based on
    # previously and add the values to our lists
    x_data.append([char_to_num[char] for char in in_seq])
    y_data.append(char_to_num[out_seq])

In [12]:
n_patterns = len(x_data)
print ("Total Patterns:", n_patterns)


Total Patterns: 2645568


In [13]:
X = numpy.reshape(x_data, (n_patterns, seq_length, 1))
X = X/float(vocab_len)


In [14]:
y = np_utils.to_categorical(y_data)


In [15]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

In [16]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [17]:
filepath = "model_weights_saved.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
desired_callbacks = [checkpoint]

In [18]:
model.fit(X, y, epochs=4, batch_size=256, callbacks=desired_callbacks)

Epoch 1/4
  536/10335 [>.............................] - ETA: 7:38:56 - loss: 2.9830

KeyboardInterrupt: 